In [68]:
#import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import ssl

In [27]:
#header for metadata
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [59]:
#get all the urls to scrape from search url
def get_all_anchor_tags(url):
  page = requests.get(url, headers=headers)#, proxies=proxies)
  content = page.content
  soup = BeautifulSoup(content)
  link_list = []
  for tag in soup.find_all('a'):
    link = tag.get("href")
    searchd = "/in/buy/projects/page/"
    if link.find(searchd) != -1:
      new_link = "https://housing.com" + link
      link_list.append(new_link)
  return link_list

#Scrape the url
def extract_data(url):
  page = requests.get(url, headers=headers)#, proxies=proxies)
  content = page.content
  soup = BeautifulSoup(content)
  #print(soup)
  result = {}
  flat_data = soup.findAll('div',attrs={'class' : 'css-j2zgcq'})
  flat_type = flat_data[0].find('h1',attrs={'class':'css-10rvbm3'})
  if flat_type is not None:
    flat_type = flat_type.text
  else:
    flat_type = "Flat type not mentioned"
  result['Flat type'] = flat_type
  key = []
  value = []
  for data in soup.findAll('section',attrs={'class' : 'css-13dph6'}):
    for i in data.findAll('div',attrs={'class' : 'css-r74jsk'}):
      key.append(i.text)
    for j in data.findAll('div',attrs={'class' : 'css-3o6ku8'}):
      value.append(j.text)
  for i in range(0,4):
    result[key[i]] = value[i]
  #print(result)
  tables = soup.find_all("table")
  table = tables[0]
  tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                          for row in table.find_all("tr")]
  for k in tab_data:
    result[k[0]] = k[1]
  #print(result)

  about = soup.find('div',attrs={'class' : 'css-1tetu0c'})
  #print(about.text)
  about_data = soup.find('div',attrs={'class' : 'about-text css-1d1e0rh'})
  #print(about_data.text)
  result[about.text] = about_data.text

  special_high = soup.find('div',attrs={'class' : "css-1o20zr1"})

  if special_high is not None:
    high = special_high.find('span',attrs={'class' : "css-xms1sc"}).text
    lis = []
    for i in special_high.findAll('div',attrs={'class' : "highlight css-1byt3mr"}):
      lis.append(i.text)
    result[high] = ",".join(lis)
  #print(result)
  new_dic = {}
  for k, v in result.items():
    lis=[]
    if v.find("}") != -1:
      lis.append(v.split("}")[-1])
    elif v.find("₹") != -1:
      lis.append(v.split("₹")[-1])
    else:
      lis.append(v)
    
    new_dic[k] = lis
  df = pd.DataFrame(new_dic)
  return df

In [46]:
#Write the data to the csv file
def write_to_file(dataframe):
  dataframe.to_csv('my_csv.csv', mode='w',header=True,index=False)


In [39]:
#merge the new and existing data in the csv file
def process_data_frame(df):
  try:
    old_df = pd.read_csv("my_csv.csv")
    new_df = old_df.append(df)
  except:
    new_df = df
  return new_df

In [69]:
#urls of the search page
url_list = [
          "https://housing.com/in/buy/searches/P3te34s1zvaong5td",
          "https://housing.com/in/buy/searches/P2q1uwqz5uo79lvmy",
          "https://housing.com/in/buy/searches/P6w6rzu8yq0jutxih",
          "https://housing.com/in/buy/searches/P1el4pqs71cc8a4oq",
          "https://housing.com/in/buy/searches/Presjxke82ehhai1",
          "https://housing.com/in/buy/searches/P2p755bijb3w8veen",
          "https://housing.com/in/buy/searches/P6mmiqcl6ho6er3oc",
          "https://housing.com/in/buy/searches/E4ff9",
          "https://housing.com/in/buy/searches/Pv26iopup2rxged3",
          "https://housing.com/in/buy/searches/E4e7y",
          "https://housing.com/in/buy/searches/E4di3",
          "https://housing.com/in/buy/searches/P2hln47z1q71v8uqk"
          "https://housing.com/in/buy/searches/P1jlgdrtnm1iwir8c",
          "https://housing.com/in/buy/searches/P26nrckh1cj7f6fhh"

]
for url in url_list:
  # Ignore SSL certificate errors
  ctx = ssl.create_default_context()
  ctx.check_hostname = False
  ctx.verify_mode = ssl.CERT_NONE
  #get all the url from search result
  list_of_urls = get_all_anchor_tags(url)
  print(list_of_urls)
  for i in list_of_urls:
    try:
      df = extract_data(i)
      new_df = process_data_frame(df)
      write_to_file(new_df)
    except:
      continue
#print the data
read_data = pd.read_csv("my_csv.csv");
read_data.head()  




['https://housing.com/in/buy/projects/page/144285-rohan-ananta-phase-iii-by-rohan-builders-and-developers-private-limited-in-tathawade', 'https://housing.com/in/buy/projects/page/208597-paranjape-schemes-orion-15-16-17-by-paranjape-schemes-construction-ltd-in-hinjewadi', 'https://housing.com/in/buy/projects/page/199094-kohinoor-coral-by-kohinoor-group-in-hinjewadi', 'https://housing.com/in/buy/projects/page/96684-kunal-iconia-phase-iv-by-kunal-group-in-mamurdi', 'https://housing.com/in/buy/projects/page/6147-goldfinger-tamara-by-gold-finger-kataria-developments-in-rahatani', 'https://housing.com/in/buy/projects/page/201975-godrej-hillside-1-by-godrej-properties-ltd-in-mahalunge', 'https://housing.com/in/buy/projects/page/228943-godrej-forest-grove-at-godrej-park-greens-by-godrej-properties-ltd-in-mamurdi', 'https://housing.com/in/buy/projects/page/225672-metro-life-maxima-residences-by-rama-group-in-tathawade', 'https://housing.com/in/buy/projects/page/171115-park-district-by-pride-pur

,Flat type,Build Up Area,Avg. Price,Age of property,Possession status,Project Name,Brokerage,Price,Carpet Area,Bedrooms,Bathrooms,Power Backup,Water Supply,Added,About Property,Overview,Configurations,Possession Starts,Sizes.css-1u2cvf2::after{content:'\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;},Project Size,Project Area,Launch Date,Rera Id,Pride Kingsbury Phase II Overview,Rohan Ananta Phase III Overview,Paranjape Schemes Orion 15 16 17 Overview,Kohinoor Coral Overview,Kunal Iconia Phase IV Overview,Possession Status,GoldFinger Tamara Overview,Godrej Hillside 1 Overview,Godrej Forest Grove At Godrej Park Greens Overview,Metro Life Maxima Residences Overview,Park District Overview,Vilas Javdekar Yashone Hinjawadi Phase 1 Overview,Godrej Elements Overview,Godrej Nurture Overview,Rohan Ananta Phase II Overview,VTP Blue Waters Overview,Antheia Overview,...,DR Destination Ankur Overview,Kirti Destination Ankur A And B Building Overview,Jayhind Icon Overview,Maitri Aarambh Overview,Enkay Coral Park Overview,Lunkad Anand Tarang C Building Overview,Shri Laxmi Morya Hills Overview,Gada Nithyam Phase II Overview,Maheshwari Suvarna Nagari Overview,Aakar Indigo Phase 2 Overview,Yashada Splendid Park Overview,Hillside Residency Overview,Tanish Bhakti Overview,Royal Shiv Vihar Overview,Bloom Residency C Building Overview,Tanish Srushti Phase III Overview,NKG Kalpataru Overview,Ganesh Gods County C Wing Overview,Ganesh Gods County A Wing Overview,Mehta Shree Jalaram Park Overview,PS Swagat A Overview,DR Gavhane Destinations Osiyan Overview,Vishwa Swami Krupa Residency Overview,Star Radhe Harmony Overview,Sharayu Galaxy Apartment Overview,Shriram Ramsmruti Overview,Ravitej Overview,Ravikiran Overview,Bhandari Associates Swaraj Phase IV TUV Overview,The Pride World City Overview,Pride Kingsbury Phase I Overview,Millennium Manor Overview,Maheshwari Suvarn Aalay Overview,Hicons Sea Crest Overview,Sea Breeze Apartment Overview,Raheja Raheja Complex Overview,harsha 2 Overview,Rajat Rekha CHS Overview,Shreedham Splendour Overview,Zee Sahyadri Overview
0,2 BHK Apartment,950 sq.ft,3.47 K/sq.ft,1 Years,Ready to move in,Jhamtani Ace Aastha Building B by Jhamtani Group,No Charge,33.0 L,592 sq.ft,2.0,3.0,Full,Bore and Municipal water,More than a month ago,2 BHK Apartment available for sale in Charholi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2 BHK Apartment,950 sq.ft,3.47 K/sq.ft,1 Years,Ready to move in,Jhamtani Ace Aastha Building B by Jhamtani Group,No Charge,33.0 L,592 sq.ft,2.0,3.0,Full,Bore and Municipal water,More than a month ago,2 BHK Apartment available for sale in Charholi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pride Kingsbury Phase II,NaN,6.61 K/sq.ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kingsbury Phase II is a brand new magnificent ...,NaN,"2, 3 BHK Apartments","Jun, 2023",579.00 sq.ft. - 783.00 sq.ft.,4 Buildings - 319 units,2 Acres,"Apr, 2019",P52100019148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pride Kingsbury Phase II,NaN,6.61 K/sq.ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kingsbury Phase II is a brand new magnificent ...,NaN,"2, 3 BHK Apartments","Jun, 2023",579.00 sq.ft. - 783.00 sq.ft.,4 Buildings - 319 units,2 Acres,"Apr, 2019",P52100019148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na